In [1]:
# first change the directory over to ssbd release
import os 
import xml.etree.ElementTree as ET

In [2]:
# next step would be to get the hand flapping and spinning data 

tree = ET.parse("ssbd-release/Annotations/v_ArmFlapping_07.xml")
root = tree.getroot()
for child in root:
    # for each child in the root 
    if child.tag == "url":
        print(child.text)
    if child.tag == "behaviours":
        for behavior in child: # go through each reported behavior 
            for tag in behavior: # tag is just the attribute of the behavior 
                if tag.tag == "time":
                    print(tag.text)
                if tag.tag == "intensity":
                    print(tag.text)
                if tag.tag == "category":
                    print(tag.text)

http://www.youtube.com/watch?v=T9rbit_oiJA
0004:0010
armflapping
high
0015:0020
armflapping
high


In [3]:
def convert_to_second(time : str) -> int:
    # this will take in a time like "0125" or 1:25 and make it 85 (60 + 25)
    overall_seconds = 0 
    for i, time_char in enumerate(reversed(time)):
        if i == 0:
            overall_seconds += int(time_char)
        if i == 1:
            overall_seconds += int(time_char) * 10
        if i == 2:
            overall_seconds += int(time_char) * 60 
        if i == 3:
            overall_seconds += int(time_char) * 600 
    return overall_seconds 

assert convert_to_second('2345') == 23 * 60 + 45  

In [4]:
from collections import defaultdict
URLS_TO_DOWNLOAD = set() # contains all youtube videos to download 
links_to_times = {} # data will be stored here like {link : {'category' : [(start, end)], 'another cat' : [(start, end)]}

for i, file_name in enumerate(os.listdir('ssbd-release/Annotations/')):
    # parse this file 
    tree = ET.parse('ssbd-release/Annotations/' + file_name)
    root = tree.getroot() 
    
    # everything we need to store
    URL = ""
    
    for child in root:
        
        if child.tag == "url":
            URL = child.text # store URL
            URLS_TO_DOWNLOAD.add(URL)
            links_to_times[URL] = defaultdict(list) 
        
        if child.tag == "behaviours": # this child has the list of behaviors 
            for reported_behavior in child: 
                for info in reported_behavior:
                    # gather the start time, end time, and category for this youtube link 
                    if info.tag == "time":
                        # the time will be start:end 
                        times = str(info.text) # contains the string 
                        if times.count(":"): 
                            divider_index = times.index(":")
                        elif times.count("-"):
                            divider_index = times.index("-")
                        else:
                            break # invalid then 
                        START_TIME, END_TIME = convert_to_second(times[:divider_index]), convert_to_second(times[divider_index + 1:])
                        
                    if info.tag == "category":
                        # this is the label 
                        LABEL = info.text 
                
                # create an entry for this reported behavior 
                links_to_times[URL][LABEL].append((START_TIME, END_TIME))

In [5]:
# pickle this just incase 
import pickle 
with open("links_to_times.pkl", 'wb') as f:
    pickle.dump(links_to_times, f)

In [29]:
# nice so the next thing to do would be to download the links 
url = "http://www.youtube.com/watch?v=I38WtUMuQwM"
from pytube import YouTube
y = YouTube("http://www.youtube.com/watch?v=I38WtUMuQwM")
y.streams.get_highest_resolution().download()
y.title 

'Just two little autistic kids... :-)'

In [28]:
import cv2
import numpy as np
from pytube import YouTube # used to download the videos
FPS = 30 
for url, category_times in links_to_times.items(): 
    
    # download the video 
    y = YouTube(url)
    y.streams.get_highest_resolution().download() 
    
    # store all frames in an array
    
    frames = [] # frames of the video 
    cap = cv2.VideoCapture(y.title + ".mp4")
    
    
    
    # get all frames for whatever category and write them to an mp4
    
    # delete the video 